In [1]:
from bs4 import BeautifulSoup
import requests
import datetime as dt
from PIL import Image
import helpers as hf
import yaml
from unidecode import unidecode
import pandas as pd
from PIL import Image, ImageOps, ImageDraw, ImageFont
import json
import numpy as np


with open("design_config.yml", "r") as f:
    design_config = yaml.safe_load(f)

design_config,shop_config = hf.get_config('cbb')
cfbd_loader = hf.cfbp_handler(design_config['cfbd_api'])

In [2]:
teams = cfbd_loader.get_team_info()

In [3]:
teams.loc[teams.name == 'Xavier']

,name,abrev,alt_color,color,division,id,logos,long_name,mascot,team_id
1811,Xavier,XU,None,#0C2340,None,1000064.0,https://1000logos.net/wp-content/uploads/2021/...,None,Musketeers,NaN


In [13]:
def cbb_cfb_exceptions(df):
    """
    there are some that dont match up from one source to the other, findd and try to fix
    """
    changes = {'Miami (FL)':'Miami'}

    df = df.replace({"team1": changes})
    df = df.replace({"team2": changes})
    return df


In [16]:
# testing parenthesis in PIL
import PIL
from PIL import Image, ImageFont, ImageDraw

im1=Image.open('art/cover3.jpeg')

# Drawing the text on the picture
font = ImageFont.truetype('fonts/Freshman.ttf', 100)
draw = ImageDraw.Draw(im1)
draw.text((50, 200), 'test(1)', (255,0,255), font=font)

# Save the image with a new name
im1.show()

In [2]:
bb = hf.get_day_bball_games(date_=pd.to_datetime('2023-04-01'))

In [3]:
bb

,date_,team1,team1Rank,team1Score,team2,team2Rank,team2Score,Desc
0,2023-04-01,UConn,(4),72,Miami (FL),(5),59,"NCAA Men's Tournament, National - National Sem..."
1,2023-04-01,San Diego State,(5),72,Florida Atlantic,(9),71,"NCAA Men's Tournament, National - National Sem..."


In [14]:
cbb_cfb_exceptions(bb)

,date_,team1,team1Rank,team1Score,team2,team2Rank,team2Score,Desc
0,2023-04-01,UConn,(4),72,Miami,(5),59,"NCAA Men's Tournament, National - National Sem..."
1,2023-04-01,San Diego State,(5),72,Florida Atlantic,(9),71,"NCAA Men's Tournament, National - National Sem..."


In [32]:
big_body = """
{
  product(id:"gid://shopify/Product/prod_id") {
    title
    media(first:5) {
      edges {
        node {
          ... fieldsForMediaTypes
        }
      }
    }
  }
}

fragment fieldsForMediaTypes on Media {
  alt
  mediaContentType
  preview {
    image {
      id
      altText
      url
    }
  }
  status
  ... on Video {
    id
    sources {
      format
      height
      mimeType
      url
      width
    }
    originalSource {
      format
      height
      mimeType
      url
      width
    }
  }
  ... on ExternalVideo {
    id
    host
    embeddedUrl
  }
  ... on Model3d {
    sources {
      format
      mimeType
      url
    }
    originalSource {
      format
      mimeType
      url
    }
  }
  ... on MediaImage {
    id
    image {
      altText
      url
    }
  }
}
"""

big_body2 = """
{
  product(id:"gid://shopify/Product/prod_id") {
    title
    media(first:10) {
      edges {
        node {
        }
      }
    }
  }
}
"""
mid_body="""{
"query": "mutation productReorderMedia($id: ID!, $moves: [MoveInput!]!) { productReorderMedia(id: $id, moves: $moves) { job { id } } }",
 "variables": {
    "id": "gid://shopify/Product/prod_id",
    "moves": [
      {
        "id": "image0",
        "newPosition": "2"
      },
      {
        "id": "image2",
        "newPosition": "0"
      }
    ]
  }
}"""


In [4]:
headers_shopify = {
            "X-Shopify-Access-Token": shop_config["shopify_access"],
            "Content-Type": "application/json",
        }

In [29]:
def recursive_get_prods(since_id=0,all_products=[]):
    url = f"https://{shop_config['cbb']['shop_name']}.myshopify.com/admin/api/2023-01/products.json?since_id={since_id}"
    response = requests.get(url=url,headers = headers_shopify)
    products = json.loads(response.text)['products']
    print(response.status_code)
    if len(products)==0:
        return all_products
    else:
        # new products were found, there could be more in the next iteration
        all_products.extend(products)
        return recursive_get_prods(since_id=products[-1]["id"],all_products=all_products)

In [72]:
all_prods = recursive_get_prods(since_id=0,all_products=[])

200
200


In [100]:
dif = (pd.to_datetime('2023-12-25')- dt.datetime.now())
seconds_in_day = 24 * 60 * 60
dif_s = (int(dif.days) * int(seconds_in_day))+ dif.seconds
hours = dif_s/(60*60)
hours

-828.2219444444445

In [74]:
all_prods[-1]

{'id': 8350548558045,
 'title': 'Tester',
 'body_html': 'The unisex heavy cotton tee is the basic staple of any wardrobe. It is the foundation upon which casual fashion grows. All it needs is a personalized design to elevate things to profitability. The specially spun fibers provide a smooth surface for premium printing vividity and sharpness. No side seams mean there are no itchy interruptions under the arms. The shoulders have tape for improved durability.<br><p>.: Made with medium fabric (5.3 oz/yd² (180 g/m²)) consisting of 100% cotton for year-round comfort that is sustainable and highly durable. <br>.: The classic fit of this shirt ensures a comfy, relaxed wear while the crew neckline adds that neat, timeless look that can blend into any occasion, casual or semi-formal.<br>.: The tear-away label means a scratch-free experience with no irritation or discomfort whatsoever.<br>.: Made using 100% US cotton that is ethically grown and harvested. Gildan is also a proud member of the US

In [71]:
pd.to_datetime(all_prods[0]['published_at']).date()

datetime.date(2024, 1, 27)

In [70]:
pd.to_datetime(all_prods[0]['updated_at']).date()

datetime.date(2024, 1, 28)

In [33]:
for prod in all_prods:
    prod_id = prod['id']
    prod_gql = prod['admin_graphql_api_id']
    body = big_body.replace('prod_id',str(prod_id))
    # now that I have the images I can reorder them
    url = f"https://{shop_config['cbb']['shop_name']}.myshopify.com/admin/api/2024-01/graphql.json"
    # from this response i can get my images
    response = requests.post(url=url,headers = headers_shopify, json = {"query":body})

    images = json.loads(response.text)['data']['product']['media']['edges']
    image0 = images[0]['node']['id']
    image2 = images[2]['node']['id']

    # image0 = prod['images'][0]['admin_graphql_api_id']
    # image2 = prod['images'][2]['admin_graphql_api_id']

    query = """
    mutation productReorderMedia($id: ID!,
    $moves: [MoveInput!]!) {
      productReorderMedia(id: $id, moves:
      $moves) {
        job {
        id
        }
      }
    }"""


    # plug my images and id into this and it should update
    variables = {
        "id": f"{prod_gql}",
        "moves": [
            {
            "id": f"{image0}",
            "newPosition": "2"
            },
            {
            "id": f"{image2}",
            "newPosition": "0"
            }
        ]
        }
    response2 = requests.post(url=url, json={'query': query , 'variables': variables}, headers= headers_shopify)



    if response2.status_code == 200:
        print(response2.text)

{"data":{"productReorderMedia":{"job":{"id":"gid://shopify/Job/12072c7d-6726-42d8-aee8-1e437d7afba8"}}},"extensions":{"cost":{"requestedQueryCost":10,"actualQueryCost":10,"throttleStatus":{"maximumAvailable":2000.0,"currentlyAvailable":1990,"restoreRate":100.0}}}}
{"data":{"productReorderMedia":{"job":{"id":"gid://shopify/Job/db922f82-fa2e-4df9-bf75-a902ab03e02c"}}},"extensions":{"cost":{"requestedQueryCost":10,"actualQueryCost":10,"throttleStatus":{"maximumAvailable":2000.0,"currentlyAvailable":1990,"restoreRate":100.0}}}}
{"data":{"productReorderMedia":{"job":{"id":"gid://shopify/Job/b8169912-e01d-46e5-9fc8-2278201cabad"}}},"extensions":{"cost":{"requestedQueryCost":10,"actualQueryCost":10,"throttleStatus":{"maximumAvailable":2000.0,"currentlyAvailable":1990,"restoreRate":100.0}}}}
{"data":{"productReorderMedia":{"job":{"id":"gid://shopify/Job/053c3dee-fe07-4488-b916-b87f617824b6"}}},"extensions":{"cost":{"requestedQueryCost":10,"actualQueryCost":10,"throttleStatus":{"maximumAvailabl

In [63]:
for prod in all_prods:
  prod_id = prod['id']
  prod_gql = prod['admin_graphql_api_id']
  body = big_body.replace('prod_id',str(prod_id))
  # now that I have the images I can reorder them
  url = f"https://{shop_config['cbb']['shop_name']}.myshopify.com/admin/api/2024-01/graphql.json"
  # from this response i can get my images
  response = requests.post(url=url,headers = headers_shopify, json = {"query":body})

  images = json.loads(response.text)['data']['product']['media']['edges']
  image0 = images[0]['node']['id']
  image2 = images[2]['node']['id']

  if image0 == image2:
    print('images are same')
  q="""
  mutation reorderProductMedia {
    productReorderMedia(
      id: "gidp",
      moves: [
      {
        id: "gid0",
        newPosition: "2"
      },
      {
        id: "gid2",
        newPosition: "0"
      }
    ]) {
      job {
        id
        done
      }
      mediaUserErrors {
        code
        field
        message
      }
    }
  }
  """.replace('gidp',prod_gql).replace('gid0',image0).replace('gid2',image2)

  # print(q)
  graphql_url =f"https://{shop_config['cbb']['shop_name']}.myshopify.com/admin/api/2023-04/graphql.json"

  response = requests.post(graphql_url, headers=headers_shopify, json={'query': q})
  print(response.text)



{"data":{"productReorderMedia":{"job":{"id":"gid://shopify/Job/fb232bf9-a9d2-4885-9074-d8e8064504d4","done":false},"mediaUserErrors":[]}},"extensions":{"cost":{"requestedQueryCost":10,"actualQueryCost":10,"throttleStatus":{"maximumAvailable":2000.0,"currentlyAvailable":1990,"restoreRate":100.0}}}}
{"data":{"productReorderMedia":{"job":{"id":"gid://shopify/Job/1c9d9ef7-9b58-4946-91ae-f9bf20bfa119","done":false},"mediaUserErrors":[]}},"extensions":{"cost":{"requestedQueryCost":10,"actualQueryCost":10,"throttleStatus":{"maximumAvailable":2000.0,"currentlyAvailable":1990,"restoreRate":100.0}}}}
{"data":{"productReorderMedia":{"job":{"id":"gid://shopify/Job/56c2df0a-6dd6-45cd-8f9d-de203eed7bc5","done":false},"mediaUserErrors":[]}},"extensions":{"cost":{"requestedQueryCost":10,"actualQueryCost":10,"throttleStatus":{"maximumAvailable":2000.0,"currentlyAvailable":1990,"restoreRate":100.0}}}}
{"data":{"productReorderMedia":{"job":{"id":"gid://shopify/Job/a45a4e4c-70bf-4c53-a0c4-8cdb6d519156","d

In [59]:
check_q='''{
"query { job(id: "gid://shopify/Job/87147bdd-eda8-4a1b-bd2c-63507e1ef074") { id done } }"
}'''
check_url = f"https://{shop_config['cbb']['shop_name']}.myshopify.com/admin/api/2024-01/graphql.json"
check_response = requests.post(graphql_url, headers=headers_shopify, json={'query': check_q})

In [55]:
response.text

'{"data":{"productReorderMedia":{"job":{"id":"gid://shopify/Job/78298459-6ec4-421a-a0c8-42c1fe5572e8","done":false},"mediaUserErrors":[]}},"extensions":{"cost":{"requestedQueryCost":10,"actualQueryCost":10,"throttleStatus":{"maximumAvailable":2000.0,"currentlyAvailable":1990,"restoreRate":100.0}}}}'